IMPORTING NECESSARY PACKAGES

In [3]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

NAMING OUR TARGET CLASSES

In [4]:
classes = ["DAISY", "DANDELION", "ROSE", "SUNFLOWER", "TULIP"]
Datadirectory = "flowers/"

TRAINING THE DATA FOR TARGET CLASSES OF EACH FLOWER 

In [5]:
training_Data = [] 

def create_training_Data():
    for varieties in classes:
        path = os.path.join(Datadirectory, varieties) 

        # SETTING THE INDEX VALUE OF EACH VARIETY AS THEIR CORREPO
        label = classes.index(varieties) 

        for img in os.listdir(path):
            try:
                img_array = cv.imread(os.path.join(path, img))

                # RESIZING THE IMAGE TO 224 X 224 AS THE PRE-TRAINED MODEL FROM TENSORFLOW HUB WAS TRAINED WITH THE SAME SIZE
                new_array = cv.resize(img_array, (224, 224))
                training_Data.append([new_array, label])

            except Exception as e: 
                pass

In [6]:
# CALLING THE FUNCTION TO CREATE OUR TRIANING DATA
create_training_Data()

In [7]:
# TOTAL SIZE OF TRAINING DATA
print(len(training_Data))

4317


SHUFFLING THE TRAINING DATA SO AS TO PREVENT THE MACHINE FROM SEQUENTIAL LEARNING

In [8]:
import random
random.shuffle(training_Data)

ADDING FEATURES AND CORRESPONDING CLASS LABLES IN TWO LISTS

In [9]:
X = []
y = []

for features, label in training_Data:
    X.append(features) 
    y.append(label)

In [10]:
X = np.array(X)
y = np.array(y)

TRAIN TEST SPLIT

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

SCALING OUR DATA 

In [12]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

TRANSFER LEARNING...

In [13]:
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

USING PRE-TRAINED DATA FROM TENSORFLOW HUB

In [14]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(feature_extractor_model, input_shape=(224, 224, 3), trainable=False) # HERE trainable=False BECAUSE WE ARE FREEZING ALREADY TRAINED WEIGHTS OF THE MODEL

In [15]:
model = tf.keras.Sequential([pretrained_model_without_top_layer, tf.keras.layers.Dense(250)])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 250)               320250    
                                                                 
Total params: 2,578,234
Trainable params: 320,250
Non-trainable params: 2,257,984
_________________________________________________________________


In [17]:
base_input = model.layers[0].input
base_output = model.layers[-1].output

In [18]:
base_output

<KerasTensor: shape=(None, 250) dtype=float32 (created by layer 'dense')>

RE-TRAINING THE PRE-TRAINED MODEL TO PRODUCE 5 CLASS LABLES AT THE LAST LAYER

In [19]:
final_output = layers.Dense(128, activation = 'relu')(base_output) ## adding (final_output).

final_output = layers.Dense(64, activation = 'relu')(final_output)

final_output = layers.Dense(5, activation='softmax')(final_output)  # 5 NEURONS IN FINAL LAYER BECAUSE 5 TARGET CLASSES IN OUR TRAINING DATA 
final_output

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense_3')>

In [20]:
new_model = keras.Model(inputs = base_input, outputs = final_output)
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_input (InputLay  [(None, 224, 224, 3)]    0         
 er)                                                             
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 250)               320250    
                                                                 
 dense_1 (Dense)             (None, 128)               32128     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                             

In [21]:
new_model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [22]:
new_model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
102/102 [==============================] - 65s 585ms/step - loss: 0.5732 - accuracy: 0.7831
Epoch 2/5
102/102 [==============================] - 54s 532ms/step - loss: 0.2968 - accuracy: 0.8928
Epoch 3/5
102/102 [==============================] - 53s 524ms/step - loss: 0.2003 - accuracy: 0.9293
Epoch 4/5
102/102 [==============================] - 55s 535ms/step - loss: 0.1434 - accuracy: 0.9459
Epoch 5/5
102/102 [==============================] - 54s 533ms/step - loss: 0.0901 - accuracy: 0.9682


EVALUATING THE NEWLY TRAINED MODEL WITH TEST IMAGES

In [23]:
new_model.evaluate(X_test_scaled,y_test)

34/34 [==============================] - 20s 535ms/step - loss: 0.4538 - accuracy: 0.8787


[0.4537661373615265, 0.8787037134170532]

SAVING OUR RE-TRAINED MODEL

In [24]:
new_model.save("ReTrainedModel.h5")

LOADING AND PERFORMING PREDICTIONS FROM OUR RE-TRAINED MODEL

In [26]:
new_model = tf.keras.models.load_model(("ReTrainedModel.h5"), custom_objects = {'KerasLayer' : hub.KerasLayer})

MANUALLY CHECKING THE PREDICTION DONE BY OUR MODEL

In [35]:
test_image = cv.imread("test_image.jpg")

In [ ]:
# CONVERTING BGR TO RGB CHANNEL TO VIEW OUR TEST IMAGE
plt.imshow(cv.cvtColor(test_image, cv.COLOR_BGR2RGB))

RESIZING AND SCALING OUR TEST IMAGE 

In [37]:
# RESIZING TO 224X224
final_image = cv.resize(test_image, (224,224))

# TURNING OUR IMAGE FROM 3D FORMAT(224, 224, 3) TO 4D FORMAT(1, 224, 224, 3) AS KERAS LAYERS ACCEPT 4-DIMENSIONS
final_image = np.expand_dims(final_image, axis=0) 

# STANDARDIZATION
final_image = final_image / 255.0

MANUAL PREDICTION USING TEST IMAGE ON OUR NEWLY TRAINED MODEL

In [ ]:
Predictions = new_model.predict(final_image)

print("The Picture is of a", classes[np.argmax(Predictions)])